# --- Day 8: I Heard You Like Registers ---

You receive a signal directly from the CPU. Because of your recent assistance with jump instructions, it would like you to compute the result of a series of unusual register instructions.

Each instruction consists of several parts: the register to modify, whether to increase or decrease that register's value, the amount by which to increase or decrease it, and a condition. If the condition fails, skip the instruction without modifying the register. The registers all start at 0. The instructions look like this:

```
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
```

These instructions would be processed as follows:

- Because a starts at 0, it is not greater than 1, and so b is not modified.
- a is increased by 1 (to 1) because b is less than 5 (it is 0).
- c is decreased by -10 (to 10) because a is now greater than or equal to 1 (it is 1).
- c is increased by -20 (to -10) because c is equal to 10.
- After this process, the largest value in any register is 1.

You might also encounter <= (less than or equal to) or != (not equal to). However, the CPU doesn't have the bandwidth to tell you what all the registers are named, and leaves that to you to determine.

What is the largest value in any register after completing the instructions in your puzzle input?

Test case from above:

In [21]:
test_cases = """b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10""".split("\n")
test_cases

['b inc 5 if a > 1',
 'a inc 1 if b < 5',
 'c dec -10 if a >= 1',
 'c inc -20 if c == 10']

So this looks like I need to parse each line and evaluate the expression using [eval](https://docs.python.org/3/library/functions.html#eval):

In [67]:
import operator
from collections import defaultdict

registers = defaultdict(int) # contains register values, defaulting to zero

ops =  {"inc": "operator.add",
        "dec": "operator.sub",
        "<": "operator.lt",
        "<=": "operator.le",
        ">": "operator.gt",
        ">=": "operator.ge",
        "==": "operator.eq",
        "!=": "operator.ne"}

def parse_line(line):
    a = line.split()
    #print(f" parsing: {a}")    
    update = ops[a[1]] + "(" + str(registers[a[0]]) + "," + str(a[2]) + ")"
    
    condition = ops[a[-2]] + "(" + str(registers[a[-3]]) + "," + a[-1] + ")"
    #print(f"action: {update}, condition: {condition}")
    
    if eval(condition):
        #print(f"updating {a[0]} with {eval(update)}")
        registers[a[0]] = eval(update)
    
for line in test_cases:
    parse_line(line)
    
registers

defaultdict(int, {'a': 1, 'b': 0, 'c': -10})

That was pretty straightforward, now to check the max value:

In [68]:
def max_value_dict(d):
    """takes in a dict d and returns the max value and keys"""
    max_value = max(d.values()) 

    # multiple registers could have the max value
    max_keys = [k for k, v in registers.items() if v == max_value]
    print(f"Max Value: {max_value}, max_keys: {max_keys}")
    return max_value, max_keys

max_value_dict(registers)

Max Value: 1, max_keys: ['a']


(1, ['a'])

In [69]:
with open('puzzle_inputs/day8_input.txt') as f:
    data = f.read().split("\n")
puzzle_input = [line for line in data if len(line)>0]
puzzle_input[:5]

['js inc 257 if wn < 9',
 'jq dec -586 if esb != -3',
 'gcf inc -603 if i >= -9',
 'gcf dec -300 if d != 1',
 'g inc -973 if gy > -1']

In [70]:
registers = defaultdict(int)

def do_puzzle(puzzle_input):
    """takes in a puzzle_input, updates registers, then returns the max value"""
    for line in puzzle_input:
        a = line.split()
        #print(f" parsing: {a}")    
        update = ops[a[1]] + "(" + str(registers[a[0]]) + "," + str(a[2]) + ")"

        condition = ops[a[-2]] + "(" + str(registers[a[-3]]) + "," + a[-1] + ")"
        #print(f"action: {update}, condition: {condition}")

        if eval(condition):
            #print(f"updating {a[0]} with {eval(update)}")
            registers[a[0]] = eval(update)
    max_value, max_keys = max_value_dict(registers)
    return max_value, max_keys

do_puzzle(puzzle_input)

Max Value: 4888, max_keys: ['jq']


(4888, ['jq'])

# --- Part Two ---

To be safe, the CPU also needs to know the highest value held in any register during this process so that it can decide how much memory to allocate to these operations. For example, in the above instructions, the highest value ever held was 10 (in register c after the third instruction was evaluated).

This is pretty straightforward, I am just putting a check in the update loop to store the max value in a variable:

In [79]:
registers = defaultdict(int)

def do_puzzle_mem_check(puzzle_input):
    """takes in a puzzle_input, updates registers, then returns the max value"""
    highest_val = None
    for line in puzzle_input:
        a = line.split()
        #print(f" parsing: {a}")    
        update = ops[a[1]] + "(" + str(registers[a[0]]) + "," + str(a[2]) + ")"

        condition = ops[a[-2]] + "(" + str(registers[a[-3]]) + "," + a[-1] + ")"
        #print(f"action: {update}, condition: {condition}")

        if eval(condition):
            # higest val code
            new_val = eval(update)
            if highest_val is None:
                highest_val = eval(update)
            elif highest_val < new_val:
                highest_val = new_val
            
            registers[a[0]] = eval(update)
    
    max_value, max_keys = max_value_dict(registers)
    
    return max_value, max_keys, highest_val

do_puzzle_mem_check(puzzle_input)

Max Value: 4888, max_keys: ['jq']


(4888, ['jq'], 7774)